## Setting up the environment

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd 

/kaggle/input/gender-statistics-labor-force-2007121722/WB_LaborForceGender_modified.csv
/kaggle/input/gender-statistics-labor-force-2007121722/727fbec5-fd06-4dfe-bb2b-44724666158e_Data.csv


## 1. Pandas SQL

In [2]:
import pandasql as psql

file = '/kaggle/input/gender-statistics-labor-force-2007121722/727fbec5-fd06-4dfe-bb2b-44724666158e_Data.csv'

table1 = pd.read_csv(file)

table1.head()

,Time,Country Name,Labor force - female,Labor force - male,Labor force - total
0,2007,Afghanistan,1029467,5090080,6119540
1,2007,Albania,528118,703384,1231498
2,2007,Algeria,1614662,8629835,10244513
3,2007,American Samoa,..,..,..
4,2007,Andorra,..,..,..


In [3]:
#filter year and country

query1 = psql.sqldf("""
                
                SELECT *
                FROM table1
                WHERE [Country Name] = 'Morocco'
                    AND
                      Time = '2022'
                    
                    """) 

query1

,Time,Country Name,Labor force - female,Labor force - male,Labor force - total
0,2022,Morocco,2946737,9585952,12532590


## 2. Sqlite

In [4]:
import pandas as pd
import sqlite3


In [5]:
file2 = '/kaggle/input/gender-statistics-labor-force-2007121722/727fbec5-fd06-4dfe-bb2b-44724666158e_Data.csv'

table2 = pd.read_csv(file2)

conn = sqlite3.connect('database2.db') #creates a DB if it doesn't exist

table2.to_sql('table2', conn)

1060

In [6]:
query2 = """SELECT *
            FROM 
                  table2
            WHERE 
                  [Country Name] = 'Morocco'
                AND
                  Time = '2022'
         """

query2_output = pd.read_sql(query2,conn)

query2_output

,index,Time,Country Name,Labor force - female,Labor force - male,Labor force - total
0,928,2022,Morocco,2946737,9585952,12532590


## 3. BigQuery

In [7]:
 #import, create object, reference DS, fetch DS, list of tables 

from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("san_francisco_bikeshare", project = "bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))

list_of_tables = [table.table_id for table in tables]

print(list_of_tables)

Using Kaggle's public dataset BigQuery integration.
['bikeshare_regions', 'bikeshare_station_info', 'bikeshare_station_status', 'bikeshare_trips']


In [8]:
query3 = """
         SELECT DISTINCT start_station_name,end_station_name
         FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
         WHERE start_station_name = "San Mateo County Center"
         """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
query3_job = client.query(query3, job_config = safe_config)

query3_result = query3_job.to_dataframe()
query3_result.head(20)

,start_station_name,end_station_name
0,San Mateo County Center,Mezes Park
1,San Mateo County Center,Broadway at Main
2,San Mateo County Center,San Mateo County Center
3,San Mateo County Center,Stanford in Redwood City
4,San Mateo County Center,Redwood City Medical Center
5,San Mateo County Center,Redwood City Public Library
6,San Mateo County Center,Redwood City Caltrain Station
7,San Mateo County Center,Kaiser Hospital
8,San Mateo County Center,Franklin at Maple
9,San Mateo County Center,Cowper at University
